In [1]:
%matplotlib inline

# Notebook summary

The purpose of this notebook is to create a set of `.tsv` files from various sources for both host and viral genomes. It is expected that the project organization follows the relative path where all initial files are located within `../Data`. 

# Imports

In [2]:
import pandas as pd
from Bio import SeqIO

import glob
import json
import numpy as np

import subprocess
import sys
sys.path.append('../../iCUB/')
import iCUB

# First setting up the very basic sequence `.tsv` files for each phage

**Use this to select the representatives from each taxa that I care about**

In [3]:
full_df = pd.read_csv('../Data/NCBI_phage_db/paper_dataset_11_2020_with_clusters.tsv', sep='\t')
full_df['cluster_representative'] = 0
full_df

,Accession,SRA_Accession,Release_Date,Species,Genus,Family,Length,Sequence_Type,Nuc_Completeness,Genotype,...,Host_order_name,Host_family_id,Host_family_name,Host_genus_id,Host_genus_name,arbitrary_cluster_id,ranking_in_cluster,cluster_representative,CDS_density,CDS_number
0,NC_050154,NaN,2020-08-13T00:00:00Z,Escherichia phage D6,NaN,Myoviridae,91159,RefSeq,complete,NaN,...,Enterobacterales,543.0,Enterobacteriaceae,561.0,Escherichia,35,1,0,0.897366,118.0
1,NC_050143,NaN,2020-08-10T00:00:00Z,Pseudomonas phage datas,Pbunavirus,Myoviridae,60746,RefSeq,complete,NaN,...,Pseudomonadales,135621.0,Pseudomonadaceae,286.0,Pseudomonas,17,27,0,NaN,NaN
2,NC_050144,NaN,2020-08-10T00:00:00Z,Pseudomonas phage Epa14,Pbunavirus,Myoviridae,65797,RefSeq,complete,NaN,...,Pseudomonadales,135621.0,Pseudomonadaceae,286.0,Pseudomonas,6,1,0,0.908035,92.0
3,NC_050145,NaN,2020-08-10T00:00:00Z,Pseudomonas phage PaGU11,Pbunavirus,Myoviridae,65554,RefSeq,complete,NaN,...,Pseudomonadales,135621.0,Pseudomonadaceae,286.0,Pseudomonas,17,24,0,NaN,NaN
4,NC_050146,NaN,2020-08-10T00:00:00Z,Pseudomonas phage Epa7,Pbunavirus,Myoviridae,65629,RefSeq,complete,NaN,...,Pseudomonadales,135621.0,Pseudomonadaceae,286.0,Pseudomonas,5,3,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9407,AF242738,NaN,2001-02-13T00:00:00Z,Lactococcus phage phiLC3,NaN,Siphoviridae,32172,GenBank,complete,NaN,...,Lactobacillales,1300.0,Streptococcaceae,1357.0,Lactococcus,36,4,0,NaN,NaN
9408,AF165214,NaN,2000-07-02T00:00:00Z,Pseudomonas virus D3,Detrevirus,Siphoviridae,56426,GenBank,complete,NaN,...,Pseudomonadales,135621.0,Pseudomonadaceae,286.0,Pseudomonas,36,3,0,NaN,NaN
9409,AF125163,NaN,1999-03-29T00:00:00Z,Vibrio virus K139,Longwoodvirus,Myoviridae,33106,GenBank,complete,NaN,...,Vibrionales,641.0,Vibrionaceae,662.0,Vibrio,14,6,0,NaN,NaN
9410,AB008550,NaN,1998-12-28T00:00:00Z,Pseudomonas virus phiCTX,Citexvirus,Myoviridae,35580,GenBank,complete,NaN,...,Pseudomonadales,135621.0,Pseudomonadaceae,286.0,Pseudomonas,72,2,0,NaN,NaN


In [4]:
# host_species_list = [562] #For building the pipeline
host_species_list = list(set(full_df['Host_species_id']))
print(host_species_list)

[1280, 1665, 644, 2055, 1423, 104336, 1428, 666, 1307, 1308, 670, 287, 29471, 1313, 1314, 1311, 552, 1833, 1708203, 305, 562, 306, 76594, 1334, 1771959, 1464, 573, 317, 197, 1351, 1358, 1747, 470, 599, 36822, 996, 28901, 1639, 1772, 623, 1396, 1911]


**This will take quite a while to run, it's not the most efficient thing in the world but does the trick**

In [ ]:
for taxon_id in host_species_list:
    print('TAXA:', taxon_id)
    #Select the dataframe relevant to the particular species under investigation
    species_df = full_df[(full_df['Host_species_id']==taxon_id)]
    print('#####', species_df.shape)
    cluster_ids = sorted(list(set(list(species_df['arbitrary_cluster_id']))))
    for new_col in ['cluster_representative', 'CDS_density', 'CDS_number']:
        full_df.at[species_df.index, new_col] = np.nan
        
    for cluster_id in cluster_ids:
        print(cluster_id)
        cluster_df = species_df[species_df['arbitrary_cluster_id']==cluster_id]
        cluster_df = cluster_df.sort_values('ranking_in_cluster', ascending=True)
        for index in cluster_df.index:
            genome_id = full_df.loc[index]['Accession']
            print(genome_id)
            genome_nt = list(SeqIO.parse('../Data/NCBI_phage_db/phage_genomes/'
                                         '{}_phage_genomes/{}.fasta'.format(taxon_id, genome_id), 'fasta'))
            assert len(genome_nt) == 1
            genome_nt = genome_nt[0]
            nt_seq_len = len(str(genome_nt.seq))
            assert nt_seq_len == full_df.loc[index]['Length']

            #Painstakingly iterate through the very large set of sequence records to find hits for
            #this particular record. I could keep this all in memory as a dictionary so that I only
            #have to iterate through it once but it would have quite a large RAM requirement
            hits = []
            all_cds = SeqIO.parse('../Data/NCBI_phage_db/'
                                      'all_complete_phage_CDSs_11_2020.fasta', 'fasta')
            for cds in all_cds:
                if genome_id in cds.id.split(':')[0]:
                    hits.append(cds)
            #Now iterate through those hits to extract out the sequence and its upstream region
            listy = []
            for hit in hits:
                reverse_complement = False
                if 'join' in hit.id: #Just skipping complex CDSs entirely
                    continue
                if 'complement(' == hit.id[:11]:
                    reverse_complement = True
                if len(hit.id.split(':')) == 2:
                    region = hit.id.split(':')[-1]
                    if region.count('<') + region.count('>') > 0:
                        print('Found an annoying region definition')
                        continue
                    if len(region.split('..')) == 2:
                        start = int(region.split('..')[0])
                        if reverse_complement:
                            stop = int(region.split('..')[-1][:-1])
                        else:
                            stop = int(region.split('..')[-1])
                        assert stop > start
                    else:
                        print('Should not be here, possible bug to investigate')
                else:
                    print('Should not be here, possible bug to investigate')

                #Ensuring that what I extracted matches the CDS record before advancing
                assert str(genome_nt.seq[start-1:stop])==str(hit.seq)

                if reverse_complement:
                    if stop + 30 > nt_seq_len:
                        continue
                    strand = '-'
                    cds_seq = str(genome_nt.seq[start-1:stop].reverse_complement())
                    us_seq = str(genome_nt.seq[stop:stop+30].reverse_complement())
                else:
                    if start - 31 < 0:
                        continue
                    strand = '+'
                    cds_seq = str(genome_nt.seq[start-1:stop])
                    us_seq = str(genome_nt.seq[start-31:start-1])

                ###Add the relevant information to a growing list    
                listy.append([genome_id,
                              start, 
                              stop,
                              strand,
                              cds_seq,
                              us_seq])

            #Get basic summary stats about the record set that I found
            cds_len = 0
            for i in listy:
                cds_len += (i[2]-i[1])
            temp_density = cds_len / full_df.loc[index]['Length']
            
            if temp_density > 0.5:
                full_df.at[index, 'cluster_representative'] = 1
                full_df.at[index, 'CDS_density'] = temp_density
                full_df.at[index, 'CDS_number'] = len(listy)

                #Save the individual set
                single_df = pd.DataFrame(listy,
                                         columns = ['Genome_source', 'Start', 'Stop', 'Strand', 'CDS_seq', 'Upstream_seq'])
                single_df.to_csv('../Data/NCBI_phage_db/phage_genomes/'
                                 '{}_phage_genomes/{}.tsv'.format(taxon_id, genome_id),
                                sep='\t', index=False)
                break

In [ ]:
full_df.to_csv('../Data/NCBI_phage_db/'
               'paper_dataset_11_2020_with_clusters.tsv', sep='\t', index=False)

**Explore a bit**

In [ ]:
full_df[(full_df['Host_species_id']==taxon_id)
        & (full_df['cluster_representative']==1)].sort_values('CDS_number').head(n=30)

In [ ]:
for host_species in host_species_list:
    print('###', host_species)
    species_df = full_df[full_df['Host_species_id']==host_species]
    print(species_df.shape[0],
          max(species_df['arbitrary_cluster_id']),
          species_df[species_df['cluster_representative']==1].shape[0])

# And for each host

In [ ]:
filter_word = 'phage' #For ease, remove anything of, or pertaining to, the string 'phage'
for host_species in host_species_list:
    genome_locs = glob.glob('../Data/NCBI_phage_db/host_genomes/{}*.gb'.format(host_species))
    print(genome_locs)
#     genome_id = genome_locs[0].split('/')[-1].split('.')[0]
#     print(genome_id)
    listy = []
    for genome_loc in genome_locs:
        print(genome_loc)
        genome_nt = list(SeqIO.parse(genome_loc, 'genbank'))
        assert len(genome_nt) == 1
        genome_nt = genome_nt[0]
        genome_id = genome_nt.id
        print(genome_id)
        nt_seq_len = len(str(genome_nt.seq))
        for feature in genome_nt.features:
            if feature.type == 'CDS':
                if filter_word in feature.qualifiers['product'][0].lower():
                    continue
                try:
                    locus_id = feature.qualifiers['locus_tag'][0]
                except:
                    locus_id = ''
                try:
                    gene_id = feature.qualifiers['gene'][0]
                except:
                    gene_id = ''
                start = feature.location.start
                stop = feature.location.end
                strand = feature.location.strand
                if strand == 1:
                    strand = '+'
                elif strand == -1:
                    strand = '-'
                else:
                    print('ERROR')
                    break
                if strand == '-':
                    if stop + 30 > nt_seq_len:
                        continue
                    listy.append([genome_id,
                                  start, 
                                  stop, 
                                  strand,
                                  str(genome_nt.seq[start:stop].reverse_complement()),
                                  str(genome_nt.seq[stop:stop+30].reverse_complement()),
                                  locus_id,
                                  gene_id])
                elif strand == '+':
                    if start < 30:
                        continue
                    listy.append([genome_id,
                                  start, 
                                  stop,
                                  strand,
                                  str(genome_nt.seq[start:stop]),
                                  str(genome_nt.seq[start-30:start]),
                                  locus_id,
                                  gene_id])
                else:
                    print('MAJOR ERROR')
                    break
    if len(genome_locs) != 0:
        print(len(listy))
        single_df = pd.DataFrame(listy,
                                 columns = ['Genome_source', 'Start', 'Stop', 'Strand', 'CDS_seq', 'Upstream_seq', 'locus_tag', 'gene_id'])
        single_df.to_csv('../Data/NCBI_phage_db/'
                         'host_genomes/{}.tsv'.format(host_species),
                        sep='\t', index=False)

# Custom function definitions to add new columns

In [ ]:
def add_RBS_energy(df, energy_dict, col_name='aSD_binding',\
                   gaps=(4,10), expected_len=30, RBS_len=6):
    '''
    This function adds a ribosome binding site (RBS) energy column to the df based off of 
    pre-computed free energy vals (created with RNAcofold and stored in the corresponding 
    energy_dict. 
    
    Inputs:
        df - 
        energy_dict - 
        gaps - 
        expected_len - 
    
    Outputs:
        df - the transformed df object now containing the new column
    '''
    for index in df.index:
        upstream = df.loc[index,'Upstream_seq']
        test_string = upstream.replace('T', 'U')
        ###Ensure that the sequence is the proper expected length
        if len(test_string) != expected_len:
            continue
        ###Ensure that the sequence has no abnormal bases
        if test_string.count('A') + test_string.count('U') +\
                                    test_string.count('C') + test_string.count('G') != expected_len:
            continue
            
        ###Calculate the energy for the indicated gap offsets
        energy_list = []
        for gap in range(gaps[0],gaps[1]+1):
             energy_list.append(energy_dict[test_string[-gap - RBS_len: -gap]])

        df.at[index, col_name] = min(energy_list)        
    return df


def call_RNAfold(sequence):
    sequence = sequence.replace('T', 'U')
    MyOut = subprocess.Popen(['RNAfold', '-p', '--noPS', '--noDP', '--constraint'],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT)
    stdout, stderr = MyOut.communicate(input=str.encode(sequence))
    return stdout

def get_energy_RNAfold(stdout_string):
    '''
    '''
    temp = stdout_string.decode('utf-8') 
    mfe_line = temp.split('\n')[-5]
    mfe_val = mfe_line[mfe_line.index(' '):]
    mfe_val = mfe_val.strip().strip('()').strip()
    #
    ensemble_line = temp.split('\n')[-4]
    ensemble_val = ensemble_line[ensemble_line.index(' '):]
    ensemble_val = ensemble_val.strip().strip('[]').strip()
    return float(ensemble_val), float(mfe_val)    

def add_secondary_structure(df):
    '''
    
    '''
    for index in df.index:
        us_seq = df.loc[index]['Upstream_seq'] 
        cds_seq = df.loc[index]['CDS_seq']
        if len(cds_seq) < 90:
            continue
        beg_cds_seq = cds_seq[:60]
        if len(us_seq) == 30 and len(beg_cds_seq) == 60:
            seq = us_seq + beg_cds_seq
            rna_out = call_RNAfold(seq)
            e1, e2 = get_energy_RNAfold(rna_out)
            df.at[index, 'sec_struct'] = e1
            #
            rna_out = call_RNAfold(seq+'\n'+ribo_bound_constraint)
            e1, e2 = get_energy_RNAfold(rna_out)
            df.at[index, 'sec_struct_bound'] = e1

    return df

def add_iCUB_and_GC(df):
    '''
    '''
    for index in df.index:
        cds_seq = df.loc[index]['CDS_seq']
        if len(cds_seq) == 0:
            continue
        if len(cds_seq) != cds_seq.count('A') + cds_seq.count('T') + cds_seq.count('C') + cds_seq.count('G'):
            continue
        if len(cds_seq)%3 != 0:
            continue
        df.at[index, 'iCUB'] = iCUB.iCUB_Calculator(cds_seq).get_iCUB()
        #
        df.at[index, 'GC_cds'] = (cds_seq.count('G') + cds_seq.count('C')) / len(cds_seq)
        #
        us_seq = df.loc[index]['Upstream_seq']
        if len(us_seq) == 30:
            df.at[index, 'GC_upstream'] = (us_seq.count('G') + us_seq.count('C')) / len(us_seq)
    return df


def common_cleaning(df):
    df = df.reset_index(drop=True)
    df = df[df['Upstream_seq'].isnull()==False]
    df = df[df['CDS_seq'].isnull()==False]
    df = df[df['iCUB'].isnull()==False]
    df = df[df['GC_cds'].isnull()==False]
    df = df[df['GC_upstream'].isnull()==False]
    df = df[df['aSD_binding'].isnull()==False]
    df = df[df['sec_struct'].isnull()==False]
    return df

In [ ]:
sep = '\t'
upstream_len = 30

with open('../Data/energy_files/energyRef_CCUCCU_ensemble_noneConstraint.json', 'r') as infile:
       energy_dict = json.load(infile)


ribo_bound_constraint = ('.'*16) + ('x'*28)+ ('.'*46)
assert len(ribo_bound_constraint)==90

# Extending and cleaning `.tsv` files for host genomes

**Adding relevant columns for ribosome binding sites, secondary structure, codon usage bias, and GC content. This whole process takes 5-10 minutes per genome. Could definitely be optimized but for now not a bottleneck, just anticipate a few hours for this cell to run.**

In [ ]:
host_species_list

In [ ]:
host_genome_dir = '../Data/NCBI_phage_db/host_genomes/'
for host_id in host_species_list:
    print(host_id)
    if host_id == 562:
        continue
    try:
        host_df = pd.read_csv(host_genome_dir + '{}.tsv'.format(host_id), sep='\t')
    except FileNotFoundError:
        print('No genome with id:', host_id)
        continue
    ###Adds the ribosome binding site energy column
    host_df = add_RBS_energy(host_df, energy_dict, col_name='aSD_binding', gaps=(4,10))

    ###Adds the secondary structure column
    host_df = add_secondary_structure(host_df)
    
    ###Add codon usage bias column
    host_df = add_iCUB_and_GC(host_df)
    
    ###Clean things up
    host_df = common_cleaning(host_df)
    
    host_df['Start_accessibility'] = host_df['sec_struct'] - host_df['sec_struct_bound']
    
    ###Writes to a file
    host_df.to_csv(host_genome_dir + '{}.clean.tsv'.format(host_id), sep=sep, index=False)

**Explore a bit!**

In [ ]:
host_df

# Extending and cleaning `.tsv` files for viral genomes

In [ ]:
host_species_list

In [ ]:
base_viral_genome_dir = '../Data/NCBI_phage_db/phage_genomes/'

for host_id in host_species_list:
    print('#####', host_id)
    if host_id == 562:
        continue
    species_df = full_df[(full_df['Host_species_id']==host_id) & (full_df['cluster_representative']==1)]
    for index in species_df.index:
        virus_id = species_df.loc[index]['Accession']
        virus_file = base_viral_genome_dir+'{}_phage_genomes/{}.tsv'.format(host_id, virus_id)
        print(virus_file)
        viral_df = pd.read_csv(virus_file, sep='\t')
        print(viral_df.shape)
        ###Adds the ribosome binding site energy column
        viral_df = add_RBS_energy(viral_df, energy_dict, col_name='aSD_binding', gaps=(4,10))

        ###Adds the secondary structure column
        viral_df = add_secondary_structure(viral_df)

        ###Add codon usage bias column
        viral_df = add_iCUB_and_GC(viral_df)
    
        ###Clean things up
        viral_df = common_cleaning(viral_df)
            
        viral_df['Start_accessibility'] = viral_df['sec_struct'] - viral_df['sec_struct_bound']
        
        ###Writes to a file
        viral_df.to_csv(virus_file.replace('.tsv', '.clean.tsv'), sep=sep, index=False)
        print(viral_df.shape)

# Scratch

**Side analysis just to make sure that all these genomes use the standard translation table**

In [ ]:
for host_gb in glob.glob('../Data/NCBI_phage_db/host_genomes/*.gb'):
    print(host_gb)
    translation_tables = []
    genome_nt = list(SeqIO.parse(host_gb, 'genbank'))[0]
    for feature in genome_nt.features:
        if feature.type == 'CDS':
            translation_tables.append(feature.qualifiers['transl_table'][0])
    print(set(translation_tables), len(translation_tables))